In [1]:
import pandas as pd
import numpy as np
import pickle
import json

In [2]:
with open("scraped_img_urls.pkl", "rb") as f:
    scraped_img_urls = pickle.load(f)
with open("cuisine_lst.pkl", "rb") as f:
    cuisine_lst = pickle.load(f)
    cuisine_lst = [cuisine.lower() for cuisine in cuisine_lst]
    cuisine_map = {cuisine: i for i, cuisine in enumerate(cuisine_lst)}
with open("food_data/ingr_map.pkl", "rb") as f:
    ingr_map_df = pickle.load(f)

In [3]:
recipe_df = pd.read_csv("recipe_df_178k_with_updated_users.csv", 
                        converters={"ingredients": eval, "quantities": eval, 
                                    "measurement_units": eval, "tags": eval,
                                   "nutrition": eval, "steps": eval, })
user_df = pd.read_csv("users.csv")
raw_interactions_df = pd.read_csv("interactions_with_updated_users.csv")
pp_recipe_df = pd.read_csv("food_data/PP_recipes.csv", converters={"ingredient_ids": eval})

In [4]:
recipe_df["calories"] = recipe_df["nutrition"].str[0]

In [5]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178265 entries, 0 to 178264
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               178265 non-null  object 
 1   id                 178265 non-null  int64  
 2   minutes            178265 non-null  int64  
 3   contributor_id     178265 non-null  int64  
 4   submitted          178265 non-null  object 
 5   tags               178265 non-null  object 
 6   nutrition          178265 non-null  object 
 7   n_steps            178265 non-null  int64  
 8   steps              178265 non-null  object 
 9   description        174311 non-null  object 
 10  ingredients        178265 non-null  object 
 11  n_ingredients      178265 non-null  int64  
 12  food_recipe_url    178265 non-null  object 
 13  quantities         178265 non-null  object 
 14  measurement_units  178265 non-null  object 
 15  calories           178265 non-null  float64
dtypes:

In [6]:
display(recipe_df.iloc[0].ingredients)
display(recipe_df.iloc[0].quantities)
display(recipe_df.iloc[0].measurement_units)
display(recipe_df.iloc[0].tags)

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

['1', '1 -2', '1 -2', '', '', '', '']

['lb', 'teaspoon', 'teaspoon', '', '', '', '']

['60-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'side-dishes',
 'vegetables',
 'mexican',
 'easy',
 'fall',
 'holiday-event',
 'vegetarian',
 'winter',
 'dietary',
 'christmas',
 'seasonal',
 'squash']

## Converting Strings to Numbers. Three Cases
1. Fractions ("1/3", "1/2" --> 0.333, 0.5)
2. Ranges ("1 -2" --> 1)
3. Normal integers ("1", "4" --> 1, 4)
4. Range with Fractions ("1⁄4 - 1⁄2" --> 1/4)

In [7]:
def convert_to_float(frac_str):
    if frac_str == "":
        return np.nan
    if " " in frac_str:
        frac_str = frac_str.split(" ")[0]
    try:
        return float(frac_str)
    except ValueError:
        if "⁄" in frac_str:
            split_frac = frac_str.split("⁄")
            num, denom = split_frac[0], split_frac[1]
            float_str = float(num) / float(denom)
        return float_str

In [8]:
recipe_df["quantities"] = recipe_df.quantities.map(lambda x: [convert_to_float(item) for item in x])
recipe_df["measurement_units"] = recipe_df["measurement_units"].map(
    lambda x: np.array([item if item != "" else np.nan for item in x], dtype=object)
)

In [9]:
recipe_df["tags"].head()

0    [60-minutes-or-less, time-to-make, course, mai...
1    [30-minutes-or-less, time-to-make, course, mai...
2    [60-minutes-or-less, time-to-make, course, mai...
3    [15-minutes-or-less, time-to-make, course, mai...
4    [15-minutes-or-less, time-to-make, course, mai...
Name: tags, dtype: object

In [ ]:
recipe_df["tags"].str.repl

In [10]:
def output_tag(tag_lst):
    lst = [tag for tag in tag_lst for cuisine in cuisine_map if (cuisine in tag)]
    if len(lst) == 0:
        return "global"
    return lst[0]

In [11]:
recipe_df["cuisine"] = recipe_df.tags.map(output_tag)

In [12]:
recipe_df["img_url"] = scraped_img_urls

In [13]:
recipe_df["serves"] = np.random.choice(np.arange(1, 5), size=recipe_df.shape[0])

In [14]:
recipe_df.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients', 'food_recipe_url', 'quantities', 'measurement_units',
       'calories', 'cuisine', 'img_url', 'serves'],
      dtype='object')

In [15]:
recipe_df = recipe_df.rename(columns={"id": "recipe_id", "minutes": "time_to_prepare", 
                          "submitted": "date_submitted", "steps": "recipe_text", "name": "recipe_name",
                         "contributor_id": "creator_id"})

In [16]:
recipe_df["recipe_text"] = recipe_df["recipe_text"].map(lambda x: ". ".join([elem.capitalize() for elem in x]))

In [17]:
recipe_df["recipe_text"]

0         Make a choice and proceed with recipe. Dependi...
1         Preheat oven to 425 degrees f. Press dough int...
2         Place potatoes in a large pot of lightly salte...
3         Combine ingredients in blender. Cover and blen...
4         Toast the fennel seeds and lightly crush them....
                                ...                        
178260    Into each of six sterile pint mason jars , put...
178261    Place the lettuce on a platter or serving dish...
178262    To make the sauce , combine the mayonnaise , h...
178263    Bring 3 quarts salted to water to a boil. Add ...
178264    In a bowl , combine the mashed yolks and mayon...
Name: recipe_text, Length: 178265, dtype: object

In [50]:
unique_tags = np.unique(np.concatenate(recipe_df["tags"].values))

In [55]:
unique_tags = set()
for tags in recipe_df["tags"].values:
    for tag in tags:
        unique_tags.add(tag)
        
tag_id_to_tag_map = {i: unique_tag for i, unique_tag in enumerate(unique_tags)}
tag_to_tag_id_map = {unique_tag: i for i, unique_tag in enumerate(unique_tags)}

In [18]:
cuisine_df = pd.DataFrame({"cuisine_id": np.arange(recipe_df["cuisine"].unique().shape[0]), 
                           "cuisine_name": recipe_df["cuisine"].unique()})

In [19]:
recipe_df = recipe_df.merge(pp_recipe_df, 
                left_on="recipe_id",
                            
                right_on="id")[recipe_df.columns.tolist() + ["ingredient_ids"]]
recipe_df = recipe_df[~recipe_df.index.isin(
    recipe_df[recipe_df["ingredient_ids"].str.len() != recipe_df["quantities"].str.len()].index
)]
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155751 entries, 0 to 178263
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   recipe_name        155751 non-null  object 
 1   recipe_id          155751 non-null  int64  
 2   time_to_prepare    155751 non-null  int64  
 3   creator_id         155751 non-null  int64  
 4   date_submitted     155751 non-null  object 
 5   tags               155751 non-null  object 
 6   nutrition          155751 non-null  object 
 7   n_steps            155751 non-null  int64  
 8   recipe_text        155751 non-null  object 
 9   description        152310 non-null  object 
 10  ingredients        155751 non-null  object 
 11  n_ingredients      155751 non-null  int64  
 12  food_recipe_url    155751 non-null  object 
 13  quantities         155751 non-null  object 
 14  measurement_units  155751 non-null  object 
 15  calories           155751 non-null  float64
 16  cu

In [20]:
recipe_df = recipe_df.merge(cuisine_df, 
                left_on="cuisine", 
                right_on="cuisine_name")[recipe_df.columns.tolist() + ["cuisine_id"]]

recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155751 entries, 0 to 155750
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   recipe_name        155751 non-null  object 
 1   recipe_id          155751 non-null  int64  
 2   time_to_prepare    155751 non-null  int64  
 3   creator_id         155751 non-null  int64  
 4   date_submitted     155751 non-null  object 
 5   tags               155751 non-null  object 
 6   nutrition          155751 non-null  object 
 7   n_steps            155751 non-null  int64  
 8   recipe_text        155751 non-null  object 
 9   description        152310 non-null  object 
 10  ingredients        155751 non-null  object 
 11  n_ingredients      155751 non-null  int64  
 12  food_recipe_url    155751 non-null  object 
 13  quantities         155751 non-null  object 
 14  measurement_units  155751 non-null  object 
 15  calories           155751 non-null  float64
 16  cu

In [21]:
pd.set_option('display.max_columns', None)

In [22]:
def unnesting(df, explode):
    idx = df.index.repeat(df[explode[0]].str.len())
    df1 = pd.concat([
        pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
    df1.index = idx

    return df1.join(df.drop(explode, 1), how='left').reset_index(drop=True)

In [23]:
recipe_sample_ingredients_df = unnesting(recipe_df, ["ingredients", "ingredient_ids", "quantities", "measurement_units"])

In [24]:
recipe_sample_ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366604 entries, 0 to 1366603
Data columns (total 21 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ingredients        1366604 non-null  object 
 1   ingredient_ids     1366604 non-null  int64  
 2   quantities         1281236 non-null  float64
 3   measurement_units  1278258 non-null  object 
 4   recipe_name        1366604 non-null  object 
 5   recipe_id          1366604 non-null  int64  
 6   time_to_prepare    1366604 non-null  int64  
 7   creator_id         1366604 non-null  int64  
 8   date_submitted     1366604 non-null  object 
 9   tags               1366604 non-null  object 
 10  nutrition          1366604 non-null  object 
 11  n_steps            1366604 non-null  int64  
 12  recipe_text        1366604 non-null  object 
 13  description        1336514 non-null  object 
 14  n_ingredients      1366604 non-null  int64  
 15  food_recipe_url    1366604 non-n

In [25]:
recipe_sample_ingredients_df

,ingredients,ingredient_ids,quantities,measurement_units,recipe_name,recipe_id,time_to_prepare,creator_id,date_submitted,tags,nutrition,n_steps,recipe_text,description,n_ingredients,food_recipe_url,calories,cuisine,img_url,serves,cuisine_id
0,winter squash,7933,1.0,lb,arriba baked winter squash mexican style,137739,55,1,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,Make a choice and proceed with recipe. Dependi...,autumn is my favorite time of year to cook! th...,7,https://www.food.com/recipe/arriba---baked-win...,51.5,north-american,https://img.sndimg.com/food/image/upload/w_555...,1,0
1,mexican seasoning,4694,1.0,teaspoon,arriba baked winter squash mexican style,137739,55,1,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,Make a choice and proceed with recipe. Dependi...,autumn is my favorite time of year to cook! th...,7,https://www.food.com/recipe/arriba---baked-win...,51.5,north-american,https://img.sndimg.com/food/image/upload/w_555...,1,0
2,mixed spice,4795,1.0,teaspoon,arriba baked winter squash mexican style,137739,55,1,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,Make a choice and proceed with recipe. Dependi...,autumn is my favorite time of year to cook! th...,7,https://www.food.com/recipe/arriba---baked-win...,51.5,north-american,https://img.sndimg.com/food/image/upload/w_555...,1,0
3,honey,3723,NaN,NaN,arriba baked winter squash mexican style,137739,55,1,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,Make a choice and proceed with recipe. Dependi...,autumn is my favorite time of year to cook! th...,7,https://www.food.com/recipe/arriba---baked-win...,51.5,north-american,https://img.sndimg.com/food/image/upload/w_555...,1,0
4,butter,840,NaN,NaN,arriba baked winter squash mexican style,137739,55,1,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,Make a choice and proceed with recipe. Dependi...,autumn is my favorite time of year to cook! th...,7,https://www.food.com/recipe/arriba---baked-win...,51.5,north-american,https://img.sndimg.com/food/image/upload/w_555...,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366599,garlic,3184,1.0,teaspoon,prawn salad pla gung,502817,25,321,2013-07-02,"[30-minutes-or-less, time-to-make, main-ingred...","[126.0, 2.0, 2.0, 36.0, 42.0, 1.0, 1.0]",8,"Bring water to a boil and add lemon grass , li...",this recipe is from recipes of asia for laos.\...,12,https://www.food.com/recipe/prawn-salad--pla-g...,126.0,laotian,https://geniuskitchen.sndimg.com/fdc-new/img/f...,4,29
1366600,ginger,3248,1.0,teaspoon,prawn salad pla gung,502817,25,321,2013-07-02,"[30-minutes-or-less, time-to-make, main-ingred...","[126.0, 2.0, 2.0, 36.0, 42.0, 1.0, 1.0]",8,"Bring water to a boil and add lemon grass , li...",this recipe is from recipes of asia for laos.\...,12,https://www.food.com/recipe/prawn-salad--pla-g...,126.0,laotian,https://geniuskitchen.sndimg.com/fdc-new/img/f...,4,29
1366601,black pepper,590,0.5,teaspoon,prawn salad pla gung,502817,25,321,2013-07-02,"[30-minutes-or-less, time-to-make, main-ingred...","[126.0, 2.0, 2.0, 36.0, 42.0, 1.0, 1.0]",8,"Bring water to a boil and add lemon grass , li...",this recipe is from recipes of asia for laos.\...,12,https://www.food.com/recipe/prawn-salad--pla-g...,126.0,laotian,https://geniuskitchen.sndimg.com/fdc-new/img/f...,4,29
1366602,green onion,6335,0.5,cup,prawn salad pla gung,502817,25,321,2013-07-02,"[30-minutes-or-less, time-to-make, main-ingred...","[126.0, 2.0, 2.0, 36.0, 42.0, 1.0, 1.0]",8,"Bring water to a boil and add lemon grass , li...",this recipe is from recipes of asia for laos.\...,12,https://www.food.com/recipe/prawn-salad--pla-g...,126.0,laotian,https://geniuskitchen.sndimg.com/fdc-new/img/f...,4,29


In [26]:
temp_df = recipe_sample_ingredients_df[(recipe_sample_ingredients_df["quantities"].notna()
                              & recipe_sample_ingredients_df["measurement_units"].isna())]

In [27]:
mode = lambda x: x.mode()[0] if x.mode().size > 0 else x.mode()
most_popular_measure_units = recipe_sample_ingredients_df[["ingredient_ids", "measurement_units"]].groupby(
    "ingredient_ids"
)["measurement_units"].agg(mode).to_dict()

In [28]:
temp_df["measurement_units"].fillna(temp_df["ingredient_ids"].map(most_popular_measure_units))

96              cup
355             cup
1895          flour
2237            cup
3701        unbaked
             ...   
1365579    boneless
1365935        eggs
1366091    teaspoon
1366261         lbs
1366505        plum
Name: measurement_units, Length: 2978, dtype: object

In [29]:
recipe_sample_ingredients_df.loc[recipe_sample_ingredients_df.index.isin(temp_df.index), "measurement_units"] = (
    temp_df["measurement_units"].fillna(
    temp_df["ingredient_ids"].map(most_popular_measure_units)))

In [30]:
recipe_sample_ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366604 entries, 0 to 1366603
Data columns (total 21 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ingredients        1366604 non-null  object 
 1   ingredient_ids     1366604 non-null  int64  
 2   quantities         1281236 non-null  float64
 3   measurement_units  1281236 non-null  object 
 4   recipe_name        1366604 non-null  object 
 5   recipe_id          1366604 non-null  int64  
 6   time_to_prepare    1366604 non-null  int64  
 7   creator_id         1366604 non-null  int64  
 8   date_submitted     1366604 non-null  object 
 9   tags               1366604 non-null  object 
 10  nutrition          1366604 non-null  object 
 11  n_steps            1366604 non-null  int64  
 12  recipe_text        1366604 non-null  object 
 13  description        1336514 non-null  object 
 14  n_ingredients      1366604 non-null  int64  
 15  food_recipe_url    1366604 non-n

In [31]:
recipe_sample_ingredients_df = recipe_sample_ingredients_df[["recipe_id", "ingredient_ids", "quantities", "measurement_units"]]
recipe_sample_ingredients_df = recipe_sample_ingredients_df.rename(columns={"ingredient_ids": "ingredient_id", "quantities": "quantity",
                                            "measurement_units": "measurement_unit"})

In [32]:
recipe_sample_ingredients_df.head()

,recipe_id,ingredient_id,quantity,measurement_unit
0,137739,7933,1.0,lb
1,137739,4694,1.0,teaspoon
2,137739,4795,1.0,teaspoon
3,137739,3723,NaN,NaN
4,137739,840,NaN,NaN


In [34]:
recipe_sample_ingredients_df.to_parquet("recipe_sample_ingredients_df.parquet", index=False)

ArrowTypeError: ("Expected bytes, got a 'numpy.ndarray' object", 'Conversion failed for column measurement_unit with type object')

In [47]:
recipe_df.to_csv("recipe_155k_with_all_updated_columns.csv",index=False)

In [22]:
ingr_map_df.to_csv("ingr_map.csv", index=False)

In [223]:
pp_recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178265 entries, 0 to 178264
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 178265 non-null  int64 
 1   i                  178265 non-null  int64 
 2   name_tokens        178265 non-null  object
 3   ingredient_tokens  178265 non-null  object
 4   steps_tokens       178265 non-null  object
 5   techniques         178265 non-null  object
 6   calorie_level      178265 non-null  int64 
 7   ingredient_ids     178265 non-null  object
dtypes: int64(3), object(5)
memory usage: 10.9+ MB


In [46]:
recipe_df

,recipe_name,recipe_id,time_to_prepare,creator_id,date_submitted,tags,nutrition,n_steps,recipe_text,description,ingredients,n_ingredients,food_recipe_url,quantities,measurement_units,calories,cuisine,img_url,serves,ingredient_ids,cuisine_id
0,arriba baked winter squash mexican style,137739,55,1,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,Make a choice and proceed with recipe. Dependi...,autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7,https://www.food.com/recipe/arriba---baked-win...,"[1.0, 1.0, 1.0, nan, nan, nan, nan]","[lb, teaspoon, teaspoon, nan, nan, nan, nan]",51.5,north-american,https://img.sndimg.com/food/image/upload/w_555...,1,"[7933, 4694, 4795, 3723, 840, 5006, 6270]",0
1,a bit different breakfast pizza,31490,30,2,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,Preheat oven to 425 degrees f. Press dough int...,this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6,https://www.food.com/recipe/a-bit-different--b...,"[1.0, 1.0, 3.0, 0.5, nan, 2.0]",[(10 ounce) can prepared pizza crust (or use y...,173.4,north-american,https://img.sndimg.com/food/image/upload/w_555...,4,"[5481, 6324, 2499, 4717, 6276, 1170]",0
2,apple a day milk shake,5289,0,4,1999-12-06,"[15-minutes-or-less, time-to-make, course, mai...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,Combine ingredients in blender. Cover and blen...,NaN,"[milk, vanilla ice cream, frozen apple juice c...",4,https://www.food.com/recipe/apple-a-day--milk-...,"[1.0, 1.0, 2.0, 0.5]","[cup, scoop, tablespoons, apple]",160.2,north-american,https://geniuskitchen.sndimg.com/fdc-new/img/f...,2,"[4717, 7474, 2946, 150]",0
3,aww marinated olives,25274,15,5,2002-04-14,"[15-minutes-or-less, time-to-make, course, mai...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,Toast the fennel seeds and lightly crush them....,my italian mil was thoroughly impressed by my ...,"[fennel seeds, green olives, ripe olives, garl...",9,https://www.food.com/recipe/aww--marinated-oli...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0]","[teaspoons, cup, cup, clove, teaspoon, tablesp...",380.7,north-american,https://geniuskitchen.sndimg.com/fdc-new/img/f...,1,"[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131...",0
4,better then bush s baked beans,67547,2970,9,2003-07-26,"[weeknight, time-to-make, course, main-ingredi...","[462.4, 28.0, 214.0, 69.0, 14.0, 29.0, 23.0]",9,In a very large sauce pan cover the beans and ...,i'd have to say that this is a labor of love d...,"[great northern bean, chicken bouillon cubes, ...",13,https://www.food.com/recipe/better-then-bush-s...,"[1.0, 4.0, 1.0, 0.25, 0.25, 0.25, 0.25, 1.0, 5...","[package, chicken, cups, cup, cup, cup, cup, t...",462.4,north-american,https://img.sndimg.com/food/image/upload/w_555...,2,"[3384, 1248, 2045, 4807, 1833, 5010, 3217, 488...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155746,laotian salad dressing,503418,20,1180,2013-07-07,"[30-minutes-or-less, time-to-make, course, cui...","[81.9, 5.0, 38.0, 22.0, 4.0, 2.0, 3.0]",3,Cook the sugar and water over medium heat unti...,this is a salad dressing for your tossed salad...,"[fresh lemon juice, garlic clove, fish sauce, ...",10,https://www.food.com/recipe/laotian-salad-dres...,"[0.25, 0.25, 2.0, 3.0, 0.25, 0.25, 1.0, 1.0, 1...","[cup, teaspoon, tablespoons, tablespoons, cup,...",81.9,laotian,https://img.sndimg.com/food/image/upload/w_555...,3,"[2832, 3203, 2654, 6124, 6906, 7655, 1119, 277...",29
155747,laotian steamed spicy chicken,504318,70,1750,2013-07-19,"[time-to-make, main-ingredient, cuisine, prepa...","[412.4, 25.0, 120.0, 31.0, 54.0, 43.0, 13.0]",14,To make spice paste: soak the chillies in hot ...,this recipe has a kick to it. posted for zwt9,"[chilies, salt, lemongrass, shrimp paste, garl...",11,https://www.food.com/recip